In [1]:
import moviepy.editor as mpy
import numpy as np
import cv2
import re

In [2]:
def remove_outliers(src):
    num_labels,labels,stats,centroids = cv2.connectedComponentsWithStats(src,connectivity=4,ltype=None)
    img = np.zeros((src.shape[0],src.shape[1]),np.uint8) # create a black background of all 0
    for i in range(1,num_labels):
        mask = labels == i # this step is to determine the location of the area through labels, assign labels information to the mask array, and then use the mask array as the index of img array
        if stats[i][4] > 100: 
            img[mask] = 255 # areas larger than 100 shall be painted white, and areas smaller than 100 shall be painted black
        else:
            img[mask] = 0
    return img

In [3]:
def get_contours(frm):
    # convert frame to grayscale
    frm_gray = cv2.cvtColor(frm, cv2.COLOR_BGR2GRAY)
    # remove tiny particels
    frm_out = remove_outliers(frm_gray)
    # convert frame to black and white
    frm_thresh = cv2.threshold(frm_out,0,255,cv2.THRESH_BINARY)[1]
    # extract contours
    contours,_ = cv2.findContours(frm_thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    return contours

In [4]:
def get_center(clip):
    # get first frame
    frm = clip.get_frame(t=0)
    # get the bounding box of the frame
    contours = get_contours(frm)
    # calculate the center of the first bounding box as the center of gravity of the human body
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if w > 100 and h > 100:
            center = (round((2*x+w)/2),round((2*y+h)/2))
    
    return center

In [5]:
def crop_and_resize(clip): 
    clip_x, clip_y = clip.size
    bbox = [clip_x,0,clip_y,0]
    average_h = 740
    
    # iterate through frames of the video clip
    for frm in clip.iter_frames():
        contours = get_contours(frm)
        
        # iterate over contours
        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
            # save rectangle that contains all boundingboxes inside it
            if x < bbox[0]:
                bbox[0] = x 
            if x+w > bbox[1]:
                bbox[1] = x+w 
            if y < bbox[2]:
                bbox[2] = y 
            if y+h > bbox[3]:
                bbox[3] = y+h 
    
    # crop and resize the clip
    cropped_clip = clip.fx(mpy.vfx.crop, x1=bbox[0],x2=bbox[1],y1=bbox[2],y2=bbox[3])
    resized_clip = cropped_clip.resize(height=average_h)
    
    #calculate the center of gravity of the human body in the resized clip
    center = get_center(resized_clip)
    
    return resized_clip, center

In [6]:
def change_dur(clip, duration):
    rate = clip.fps
    dur = clip.duration
    
    desired_rate = (rate*dur)/duration
    scaling_factor = desired_rate/rate

    # Modify the FPS
    clip = clip.set_fps(clip.fps*scaling_factor)
    # Apply speed up
    clip = clip.fx(mpy.vfx.speedx,scaling_factor)
    
    return clip

In [7]:
# Define which video files to load
baseline_f1 = 'IS_gg00f.mov'
baseline_f2 = 'gesture01_Female02_00000106.mov'
baseline_p = 'IS_gg00p.mov'
baseline_n = 'IS_gg00n.mov'
bow_f1 = 'gesture02_Female01_00000435.mov'
bow_f2 = 'gesture02_Female02_00000372.mov'
bow_p = 'gesture16_Pepper00000016.mov'
bow_n = 'gesture10_NAO00003079.mov'

# Define the positioning
gestures_left = [bow_f1, baseline_f1, bow_p, baseline_p]
gestures_right = [bow_f2, baseline_f2, bow_n, baseline_n]

# Calculate combinations
combis = [(left,right) for right in gestures_right for left in gestures_left]

In [8]:
for cnt, gestures in enumerate(combis):
    # Load the video files of the individual stimuli to be combined into a dyad
    if re.search(r'[NnPp]',gestures[0]):
        gesture_left = mpy.VideoFileClip(gestures[0],has_mask=True).rotate(-90.00001)
    else: 
        gesture_left = mpy.VideoFileClip(gestures[0],has_mask=True)
    
    if re.search(r'[NnPp]',gestures[1]):
        gesture_right = mpy.VideoFileClip(gestures[1],has_mask=True).rotate(-90.00001).fx(mpy.vfx.mirror_x)
    else: 
        gesture_right = mpy.VideoFileClip(gestures[1],has_mask=True).fx(mpy.vfx.mirror_x)
 
    # Transform the individual stimuli files
    gesture_right, center_right = crop_and_resize(gesture_right)
    gesture_left, center_left = crop_and_resize(gesture_left)
    
    # Speed up or slow down video to 3sec
    duration = 3
    gesture_right = change_dur(gesture_right,duration)
    gesture_left = change_dur(gesture_left,duration)
    
    # Load the background image
    background = mpy.ImageClip('background_dyads.png',duration=duration)
    
    # Create the video composition
    dyad = mpy.CompositeVideoClip([background, 
                                   gesture_left.set_position((660-center_left[0],170)), 
                                   gesture_right.set_position((1260-center_right[0],170))
                                  ])
    # Make the video grayscale
    dyad = dyad.fx(mpy.vfx.blackwhite)
    
    # Write the output video file
    dyad.write_videofile('movie' + str(cnt) + '.mp4', fps=25)

Moviepy - Building video movie0.mp4.
MoviePy - Writing audio in movie0TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video movie0.mp4



Moviepy - Done !
Moviepy - video ready movie0.mp4
Moviepy - Building video movie1.mp4.
MoviePy - Writing audio in movie1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video movie1.mp4



Moviepy - Done !
Moviepy - video ready movie1.mp4
Moviepy - Building video movie2.mp4.
MoviePy - Writing audio in movie2TEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready movie13.mp4
Moviepy - Building video movie14.mp4.
MoviePy - Writing audio in movie14TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video movie14.mp4



Moviepy - Done !
Moviepy - video ready movie14.mp4
Moviepy - Building video movie15.mp4.
MoviePy - Writing audio in movie15TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video movie15.mp4



Moviepy - Done !
Moviepy - video ready movie15.mp4
